In [8]:
import os
import pickle
import ipywidgets as widgets
from dotenv import load_dotenv
from IPython.display import display, Markdown
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

In [ ]:
load_dotenv()
openai_key = os.getenv("")

In [11]:
embedding_model = OpenAIEmbeddings()
chat = ChatOpenAI(temperature=0, openai_api_key=openai_key)

In [12]:
chat.model_name

'gpt-3.5-turbo'

In [14]:
with open("vectorstore.pkl", "rb") as f:
    index, texts, metadata = pickle.load(f)

In [17]:
qa_chain = load_qa_with_sources_chain(chat, chain_type="stuff")

query_box = widgets.Text(placeholder='Ask me anything about Mariam...')
output_box = widgets.Output()

@output_box.capture()
def handle_query(change):
    query = change["new"]
    if not query:
        return
    query_embedding = embedding_model.embed_query(query)
    D, I = index.search([query_embedding], k=4)
    source_texts = [texts[i] for i in I[0]]
    response = qa_chain({"input_documents": source_texts, "question": query}, return_only_outputs=True)

    output_box.clear_output()
    display(Markdown(f"### **Answer**\n{response['output_text']}"))
    display(Markdown("### **Sources**"))
    for i in I[0]:
        snippet = texts[i][:300].replace("\n", " ")
        display(Markdown(f"**{metadata[i]['source']}**\n> {snippet}..."))

query_box.observe(handle_query, names='value')
display(query_box, output_box)

Text(value='', placeholder='Ask me anything about Mariam...')

Output()